In [1]:
from detoxify import Detoxify
import pandas as pd
import pickle
from metrics import *

In [2]:
TEXT_COLUMN = 'comment_text'
MODEL_NAME = 'roberta-base-unbiased'
MODEL_NAME = 'roberta-base-unbiased-small'

if MODEL_NAME == 'roberta-base-unbiased':
    model = Detoxify('unbiased', device='cuda:1')
elif MODEL_NAME == 'roberta-base-unbiased-small':
    model = Detoxify('unbiased-small', device='cuda:1')

In [3]:
test_df = pd.read_csv('./Data/test_private_expanded.csv')

In [4]:
texts = test_df[TEXT_COLUMN].tolist()

In [5]:
model.model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [6]:
len(texts), texts[0:3]

(97320,
 ['[ Integrity means that you pay your debts.]\n\nDoes this apply to President Trump too?',
  'This is malfeasance by the Administrator and the Board.  They are wasting our money!',
  '@Rmiller101 - Spoken like a true elitist. But look out bud. The re-awakening in Europe, Brexit and now the Trump victory are just the beginning. The unwashed masses are coming for you and your ilk….'])

In [7]:
results = model.predict(texts[:10])
results['toxicity'][:3]

[0.0006539470632560551, 0.003939293324947357, 0.049065615981817245]

In [14]:
len(texts)
print('estimated prediction time: ', len(texts)/100*0.6/60 , 'min')

prediction time:  9.732 min


In [ ]:
predictions = []
i = 0
while i < len(texts):
    print(i, i+300)
    results = model.predict(texts[i:i+300])
    predictions.extend(results['toxicity'])
    i += 300

In [16]:
if len(predictions) == len(texts):
    print('predictions complete')
    with open(f'{MODEL_NAME}_prediction.pkl', 'wb') as f:
        pickle.dump(predictions, f)
else:
    # last batch
    print(i, len(texts))
    results = model.predict(texts[i:])
    predictions.extend(results['toxicity'])
    # save!!!
    print('predictions complete')
    with open(f'{MODEL_NAME}_prediction.pkl', 'wb') as f:
        pickle.dump(predictions, f)

predictions complete


In [15]:
with open(f'{MODEL_NAME}_prediction.pkl', 'rb') as f:
    predictions = pickle.load(f)

In [16]:
len(predictions)

97320

In [17]:
prediction_df = pd.DataFrame(predictions, columns=[MODEL_NAME])
prediction_df.head()

,roberta-base-unbiased
0,0.000583
1,0.006938
2,0.077543
3,0.001755
4,0.000604


In [18]:
test_df = test_df.assign(MODEL_NAME=prediction_df)
test_df.rename(columns={"MODEL_NAME": MODEL_NAME}, inplace=True)

In [20]:
test_df.head()

,id,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,...,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,roberta-base-unbiased
0,7097320,[ Integrity means that you pay your debts.]\n\...,2017-09-13 20:12:01.484121+00,21,5945023.0,376974,approved,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000583
1,7097321,This is malfeasance by the Administrator and t...,2017-05-17 07:01:51.902566+00,55,NaN,335003,approved,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006938
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...,2016-12-02 17:12:12.920957+00,54,649389.0,154126,approved,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077543
3,7097323,"Paul: Thank you for your kind words. I do, in...",2017-04-21 14:58:05.474657+00,13,5158666.0,328376,approved,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001755
4,7097324,Sorry you missed high school. Eisenhower sent ...,2017-10-01 19:43:12.373229+00,102,6061626.0,383983,approved,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000604


In [21]:
test_df.to_csv(f'./submissions/{MODEL_NAME}_submission.csv')